In [2]:
import requests
import pandas as pd
import re

In [3]:
url = "https://script.googleusercontent.com/macros/echo?user_content_key=CS1xGtawxafbsz_VXLceDZ8YhZJ3WLRGo3cTurMdyLFpRky5qzeXoqZBrYuJ0ElreSkNapo1i9TRXgBoQpou4xhHPHjfNUj4m5_BxDlH2jW0nuo2oDemN9CCS2h10ox_1xSncGQajx_ryfhECjZEnNGpsruz596piGbFJMaeWeZjFnhN0a9d4HwmOCyefi3ex5PPcpNsPlxGCUpGq7rwIA3gVkC6Pkbl8Md7CuyY-jH4-8QW02051A&lib=McNoStEX3P6w1EymfWUjOXqNKe58B53cM"
resp = requests.get(url)
resp_json = resp.json()

In [4]:
df = pd.DataFrame(resp_json['data'][1:], columns=resp_json['data'][0])

In [11]:
strings = df.columns
pattern = r"\[(.*?)\]"
result = []
for string in strings:
    matches = re.findall(pattern, string)
    result.extend(matches)

# Print the extracted substrings
print(result)

['Su', 'SzeKi', 'Caitlyn', 'Yau', 'Yannis', 'Yoyo', 'Heiting', 'Chuilam', 'Chautung']


In [16]:
new_columns = list(df.columns[:2]) + result
df.columns = new_columns

In [11]:
df.fillna(value = 0, inplace=True)

In [14]:
# Sample DataFrame with null values
data = {
    'Date': ['2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15'],
    'John': ['present', 'absent', None, 'present'],
    'Mary': ['absent', 'present', 'present', 'absent'],
    'Alice': ['absent', None, 'present', 'present'],
    'Bob': ['present', 'absent', 'absent', 'absent']
}

df = pd.DataFrame(data)

# Calculate present rates, present count, and absent count for all names
result = pd.DataFrame()
for column in df.columns[1:]:
    present_count = df[column].value_counts().get('present', 0)
    absent_count = df[column].value_counts().get('absent', 0)
    null_count = df[column].isnull().sum()
    total_count = present_count + absent_count
    present_rate = present_count / total_count * 100
    result.loc[column, 'Present Count'] = present_count
    result.loc[column, 'Absent Count'] = absent_count
    #result.loc[column, 'Null Count'] = null_count
    result.loc[column, 'Present Rate (%)'] = present_rate

# Print the result DataFrame
print(result)

       Present Count  Absent Count  Present Rate (%)
John             2.0           1.0         66.666667
Mary             2.0           2.0         50.000000
Alice            2.0           1.0         66.666667
Bob              1.0           3.0         25.000000


In [15]:
df

,Date,John,Mary,Alice,Bob
0,2022-02-12,present,absent,absent,present
1,2022-02-13,absent,present,None,absent
2,2022-02-14,None,present,present,absent
3,2022-02-15,present,absent,present,absent
